In [1]:
from preamble import *

In [5]:
#------------------------------------------------------------#
MaxIntFilter = 64 #knots
#------------------------------------------------------------#

In [6]:
# Load the ibtracs dataset
dataset = pd.read_csv('Data/IBTrACS/data_ibtracs_JTWC.csv')

# Apply VMAX corrections
dataset['A_VMAX'] = 0.1884 * dataset['VMAX']**1.288
dataset['VMAX2'] = 0
dataset.loc[dataset['YEAR'] < 1968, 'VMAX2'] = 0.4 * dataset['VMAX'] + 0.6 * dataset['A_VMAX']
dataset.loc[dataset['YEAR'].between(1968,1972), 'VMAX2'] = 0.8 * dataset['VMAX'] + 0.2 * dataset['A_VMAX']
dataset.loc[dataset['YEAR'] > 1972, 'VMAX2'] = dataset['VMAX']
dataset = dataset.drop(['VMAX','A_VMAX'], axis=1)
dataset = dataset.rename(columns={'VMAX2': 'VMAX'})

In [7]:
# Determine maximum intensity of the each cyclone
max_int = dataset.groupby('CycNo')['VMAX'].max().rename('VMAX').reset_index()

# Remove cyclones with maximum intensity lower than defined
max_int = max_int[max_int['VMAX'] > MaxIntFilter]
dataset = dataset[dataset['CycNo'].isin(max_int['CycNo'])]

In [9]:
dataset['CycNo'].nunique()

938

In [60]:
def influence(data):
    peak_time = test_set['S_TIME'][test_set['VMAX'].idxmax()]
    landfall_time = test_set[test_set['LANDFALL'] < 100].iloc[0]['S_TIME']
    if landfall_time <= peak_time:
        return True
    else:
        return False

In [61]:
effect_data = dataset.groupby('CycNo').apply(influence)

In [62]:
len(effect_data[effect_data == True])

0